In [1]:
%load_ext autoreload
%autoreload 2

%cd ../

/Users/macos/Uni/Thesis


In [2]:
import json
import yaml

from recbole.config import Config
from recbole.data import data_preparation, create_dataset
from recbole.trainer import HyperTuning
from recbole.utils import (
    get_model,
    get_trainer,
    init_seed
)

import src.utils as utils
from src.real_temporal import TimeCutoffDataset

# 1. Declarations & Definitions

## 1.1. Define flags and global variables

In [3]:
seed = 42

use_cutoff = True
reproducible = False

model = "BPR"
dataset = "ml-1m"
loss_type = "BPR"

## 1.2. Define configurations

Configuration for data, model, training and evaluation

In [4]:
paths = utils.Paths(model, dataset)

logger = utils.get_logger(model, dataset)

In [5]:
config_dict = {
    # For model 
    'model': model,
    'loss_type': loss_type,

    # For data
    'dataset': dataset, 
    'load_col': {"inter": ['user_id', 'item_id', 'timestamp']},
    'use_cutoff': use_cutoff,

    # For training
    'epochs': 20,
    'train_batch_size': 4096,
    'eval_step': 1,
    'stopping_step': 3,
    'learning_rate': 1e-3,
    
    # For evaluation
    'eval_batch_size': 4096,
    'metrics': ["NDCG", "Precision", "Recall", "MRR", "Hit", "MAP"],
    'topk': 10,
    'valid_metric': 'NDCG@10',

    # Environment
    'gpu_id': 0,
    "seed": seed,
    "reproducibility": reproducible,
    'device': 'mps',
    'use_gpu': True,
    'data_path': paths.get_path_data_raw(),
    "checkpoint_dir": paths.get_path_dir_ckpt(),
    "show_progress": True,
    'save_dataset': True,
    'dataset_save_path': paths.get_path_data_processed(),
    'save_dataloaders': True,
    'dataloaders_save_path': paths.get_path_dataloader(),
}

if use_cutoff is True:
    config_dict['eval_args'] = {
        "order": "TO",
        "split": {"CO": '976324045'},
        "group_by": 'user_id',
        'mode': 'full'
    }
else:
    config_dict['eval_args'] = {
        "order": "TO",
        "split": { "LS": "valid_and_test" },
        "group_by": None,
        'mode': 'full'
    }

if loss_type == "CE":
    config_dict["train_neg_sample_args"] = None
else:
    config_dict["train_neg_sample_args"] = {
        "distribution": "uniform",
        "sample_num": 1,
        # "dynamic": False,
        # "candidate_num": 0,
    }

with open(paths.get_path_conf(), 'w+') as f:
    yaml.dump(config_dict, f, allow_unicode=True)

# 2. Train

## 2.1. Declare necessary components for training

In [6]:
config = Config(config_dict=config_dict)

init_seed(config["seed"], config["reproducibility"])

# Define data related things
if use_cutoff is True:
    dataset = TimeCutoffDataset(config)
else:
    dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)

# Define model
model_name = config['model']
model = get_model(model_name)(config, train_data._dataset).to(config['device'])

# Define trainer
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

In [7]:
logger.info(config)
logger.info(dataset)
logger.info(model)

2024-07-06 00:27:02.811 | INFO     | __main__:<module>:1 - 
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 42
state = INFO
reproducibility = False
data_path = data/raw/ml-1m
checkpoint_dir = logs/Jul06_002648_BPR_ml-1m/ckpts
show_progress = True
save_dataset = True
dataset_save_path = data/processed/ml-1m.pth
save_dataloaders = True
dataloaders_save_path = data/dataloader/BPR-ml-1m.pth
log_wandb = False

Training Hyper Parameters:
epochs = 20
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 3
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'CO': '976324045'}, 'order': 'TO', 'group_by': 'user_id', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['NDCG', 'Precision', 'Recall', 'MRR', 'Hit', 'MAP']
topk = [10]
valid_metric = 

## 2.2. Start training

In [8]:
best_valid_score, best_valid_result = trainer.fit(
    train_data, 
    valid_data,
    verbose=True,
    show_progress=config["show_progress"]
)

logger.info("** Validation result")
logger.info(f"best_valid_score: {best_valid_score:.4f}")
for metric, val in best_valid_result.items():
    logger.info(f"{metric:<15}: {val:.4f}")

Evaluate   : 100%|████████████████████████████████████████████| 5682/5682 [00:01<00:00, 5300.81it/s]
2024-07-06 00:28:12.725 | INFO     | __main__:<module>:8 - ** Validation result
2024-07-06 00:28:12.725 | INFO     | __main__:<module>:9 - best_valid_score: 0.0360
2024-07-06 00:28:12.726 | INFO     | __main__:<module>:11 - ndcg@10        : 0.0360
2024-07-06 00:28:12.726 | INFO     | __main__:<module>:11 - precision@10   : 0.0073
2024-07-06 00:28:12.727 | INFO     | __main__:<module>:11 - recall@10      : 0.0725
2024-07-06 00:28:12.728 | INFO     | __main__:<module>:11 - mrr@10         : 0.0251
2024-07-06 00:28:12.728 | INFO     | __main__:<module>:11 - hit@10         : 0.0725
2024-07-06 00:28:12.729 | INFO     | __main__:<module>:11 - map@10         : 0.0251


## 3.3. Start testing

In [9]:
test_result = trainer.evaluate(test_data)

logger.info("** Test result")
for metric, val in test_result.items():
    logger.info(f"{metric:<15}: {val:.4f}")

2024-07-06 00:28:12.983 | INFO     | __main__:<module>:3 - ** Test result
2024-07-06 00:28:12.984 | INFO     | __main__:<module>:5 - ndcg@10        : 0.0369
2024-07-06 00:28:12.984 | INFO     | __main__:<module>:5 - precision@10   : 0.0075
2024-07-06 00:28:12.985 | INFO     | __main__:<module>:5 - recall@10      : 0.0748
2024-07-06 00:28:12.985 | INFO     | __main__:<module>:5 - mrr@10         : 0.0254
2024-07-06 00:28:12.986 | INFO     | __main__:<module>:5 - hit@10         : 0.0748
2024-07-06 00:28:12.987 | INFO     | __main__:<module>:5 - map@10         : 0.0254


# 4. Tune hyper params

## 4.1. Define hyper params and object function

In [10]:
def objective_function(config_dict=None, config_file_list=None):
    config = Config(
        config_dict=config_dict,
        config_file_list=config_file_list,
    )

    init_seed(config["seed"], config["reproducibility"])

    # Define data related things
    if config['use_cutoff'] is True:
        dataset = TimeCutoffDataset(config)
    else:
        dataset = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # Define model
    model_name = config['model']
    model = get_model(model_name)(config, train_data._dataset).to(config['device'])

    # Define trainer
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

    # Start training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, verbose=False)

    # Start evaluating
    test_result = trainer.evaluate(test_data)

    return {
        'model': model_name,
        'best_valid_score': best_valid_score,
        'valid_score_bigger': config['valid_metric_bigger'],
        'best_valid_result': best_valid_result,
        'test_result': test_result
    }

## 4.2. Start tuning

In [11]:
tuning_algo = "bayes"
early_stop = 3
max_evals = 5

hp = HyperTuning(
    objective_function=objective_function,
    algo=tuning_algo,
    early_stop=early_stop,
    max_evals=max_evals,
    fixed_config_file_list=[paths.get_path_conf()],
    params_file=paths.get_path_tuning_conf(),
)


hp.run()

running parameters:                                  
{'learning_rate': 0.0005}                            
current best valid score: 0.0274                     
current best valid result:                           
OrderedDict([('ndcg@10', 0.0274), ('precision@10', 0.0055), ('recall@10', 0.0547), ('mrr@10', 0.0191), ('hit@10', 0.0547), ('map@10', 0.0191)])
current test result:                                 
OrderedDict([('ndcg@10', 0.0368), ('precision@10', 0.0069), ('recall@10', 0.0689), ('mrr@10', 0.0271), ('hit@10', 0.0689), ('map@10', 0.0271)])
running parameters:                                                 
{'learning_rate': 0.0001}                                           
running parameters:                                                 
{'learning_rate': 0.005}                                            
current best valid score: 0.0429                                    
current best valid result:                                          
OrderedDict([('ndcg@10', 0.0

## 4.3. Export tunning result

In [12]:
# print best parameters
logger.info('best params: ', hp.best_params)

# print best result
logger.info('best result: ')
logger.info(hp.params2result[hp.params2str(hp.best_params)])

# export to JSON file
tune_result = {
    'best_params': hp.best_params,
    'best_result': hp.params2result[hp.params2str(hp.best_params)]
}
with open(paths.get_path_tuning_log(), "w+") as f:
    json.dump(tune_result, f, indent=2, ensure_ascii=False)

2024-07-06 00:34:27.857 | INFO     | __main__:<module>:2 - best params: 
2024-07-06 00:34:27.858 | INFO     | __main__:<module>:5 - best result: 
2024-07-06 00:34:27.859 | INFO     | __main__:<module>:6 - {'model': 'BPR', 'best_valid_score': 0.0429, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('ndcg@10', 0.0429), ('precision@10', 0.0088), ('recall@10', 0.0876), ('mrr@10', 0.0296), ('hit@10', 0.0876), ('map@10', 0.0296)]), 'test_result': OrderedDict([('ndcg@10', 0.0266), ('precision@10', 0.0053), ('recall@10', 0.0529), ('mrr@10', 0.0187), ('hit@10', 0.0529), ('map@10', 0.0187)])}
